# Load your own model

In [ ]:
from keras.models import load_model
model = load_model('model.h5')
model.summary() # to see if it is the same as previous

# Start to build your user interface using 'tkinter' 

In [ ]:
import tkinter as tk
import os
import cv2
import sys
from PIL import Image, ImageTk
import numpy

cancel1 = False
cancel2 = True

#==========================================

root = tk.Tk() # create a intergace window
root.title('NTU-EEE-DIP AI CAR by Yeqi, Yuan, Chang') # the title of the window
root.geometry('1300x600') # size of the window

canvas = tk.Canvas(root,bg='pink',height=180,width=500) # create a 'canvas' area to show some logo pictures
image_path1 = tk.PhotoImage(file='NTU.png')
image_path2 = tk.PhotoImage(file='tenor.gif')
image1 = canvas.create_image(0,0,anchor='nw',image=image_path1)
image2 = canvas.create_image(230,-20,anchor='nw',image=image_path2)
canvas.place(x=5,y=5) # the location of canvas

# divide the window into 'left' and  'right' two parts
frm_l =tk.Frame(root)
frm_r =tk.Frame(root)

frm_l.pack(side='left')
frm_r.pack(side='right')

#===================================================================
# define all the finctions will be used to react with all 'button'

def hit_me5():
    on_hit = False
    if on_hit == False:
        on_hit =True
        var.set('run forward a bit')

def hit_me6():
    on_hit = False
    if on_hit == False:
        on_hit =True
        var.set('turn left a bit')

def hit_me7():
    on_hit = False
    if on_hit == False:
        on_hit =True
        var.set('turn right a bit')

def hit_me8():
    on_hit = False
    if on_hit == False:
        on_hit =True
        var.set('stop car')

def change_cancel2():
    global cancel1,cancel2
    on_hit = False
    if on_hit == False:
        on_hit =True
        var.set('start recognition')
        cancel2 = False
        cancel1 = True
        
#===================================================================
# define the labels and buttons show in your window
# this is the left side, which is the mannual control area

ll = tk.Label(frm_l,text='Mannual control',bg='yellow',font=('Arial',18),width=15,height=3)
ll.pack()

bl1 = tk.Button(frm_l,text='run forward',width=15,height=3,command=hit_me5)
bl1.pack(side='top')

bl2 = tk.Button(frm_l,text='turn left',width=15,height=3,command=hit_me6)
bl2.pack(side='left')

bl3 = tk.Button(frm_l,text='turn right',width=15,height=3,command=hit_me7)
bl3.pack(side='right')

bl4 = tk.Button(frm_l,text='stop',width=15,height=3,command=hit_me8)
bl4.pack(side='bottom')

#===================================================================

lmain = tk.Label(root, compound=tk.CENTER, anchor=tk.CENTER, relief=tk.RAISED)
lmain.pack()

#===================================================================

# start load video and show on window
cap = cv2.VideoCapture(0) 
var = tk.StringVar()  

# this is recogniton mode function
def show_frame():
    global var,cancel2
    if cancel2 == False:
        _, frame = cap.read()
        cv2image1 = cv2.flip(frame,1)
        cv2image2 = cv2.cvtColor(cv2image1, cv2.COLOR_BGR2RGBA) # convert video to RGB format  
        
        cv2.rectangle(cv2image2,(360,60),(590,290),(0,255,0),5) # draw square
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(cv2image2,'Put your RIGHT hand in this square!',(300,40),font,0.6,(0,255,0),2,cv2.LINE_AA)
    
        crop_cv2image = cv2image1[63:287,363:587] # crop the image with size (224,224)
        img = crop_cv2image.reshape((1,)+ crop_cv2image.shape ) #reshape it so that model can accept
    
        prediction = model.predict(img)[0]                                                      
        pred_class = list(prediction).index(max(prediction))
        print(pred_class)
        
        if pred_class==1:
            var.set('run forward')
        elif pred_class==2:
            var.set('turn left a bit')
        elif pred_class==3:
            var.set('turn right')
        elif pred_class==0:
            var.set('stop')  
    
        l = tk.Label(root,textvariable=var,bg='blue',font=('Arial',22),width=25,height=5) # show instruction uder video area
        l.pack(side='bottom')
    
        prevImg = Image.fromarray(cv2image2) # this is the part shows on video area
        imgtk = ImageTk.PhotoImage(image=prevImg)
        lmain.imgtk = imgtk
        lmain.configure(image=imgtk)
        if not cancel2: # if cancel2 is False, continuously show video
            lmain.after(10, show_frame)
#===================================
# this is the main function, which is 'mannual control' mode

def ini_show_frame():
    global var,cancel1 

    _, frame = cap.read()
    cv2image1 = cv2.flip(frame,1)
    cv2image2 = cv2.cvtColor(cv2image1, cv2.COLOR_BGR2RGBA)
    
    
    cv2.rectangle(cv2image2,(360,60),(590,290),(0,255,0),5)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(cv2image2,'Put your RIGHT then click START!',(320,40),font,0.6,(0,255,0),2,cv2.LINE_AA)
    
    crop_cv2image = cv2image1[63:287,363:587]

    img = crop_cv2image.reshape((1,)+ crop_cv2image.shape ) 
    
    l = tk.Label(root,textvariable=var,bg='blue',font=('Arial',22),width=25,height=5)
    l.pack(side='bottom')
    
    prevImg = Image.fromarray(cv2image2)
    imgtk = ImageTk.PhotoImage(image=prevImg)
    lmain.imgtk = imgtk
    lmain.configure(image=imgtk)
    if cancel1 == False: # if 'start' botton is clicked, cancel1 is True
        lmain.after(10, ini_show_frame)
    else: # start recognition mode
        show_frame()
        
ini_show_frame() # call the main function

#=================================

# the 'start' botton to change the mode from 'mannual' to 'recognition'
button = tk.Button(frm_l, text="START!", width=15, height=3,command=change_cancel2) 
button.pack(side='bottom')     

#================================

root.mainloop() #*****this is the most important code to show the the window*****